In [ ]:
import pandas as pd
#from sklarn.feature_extraction.text import TfidVectorizer
from sklearn.metrics import silhouette_score


In [ ]:
df = pd.read_excel('/content/Research_Project_Dataset_2.xlsx')
df.columns

Index(['NEWS (Full Transcript)', 'Summary', ' Category', 'Unnamed: 3',
       'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6'],
      dtype='object')

In [ ]:
df['Summary']

,Summary
0,A group of persons assembles at the hospital t...
1,A 19-year-old woman was found brutally murdere...
2,The parliament case inquiry and investigation ...
3,Amin was a victim of this crime or or supposed...
4,Family of Hashi brella who was found dead in t...
...,...
195,Prime Minister Narendra Modi launched three su...
196,There is a need for at least three additional ...
197,The diamir Basha Dam is Pakistan's most expens...
198,There are reports that after the Corona virus ...


In [ ]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [ ]:
def normalize_text(text):
  text = text.lower()
  text = re.sub(r'https\S+|www\S+|http\S+', '', text, flags=re.MULTILINE)
  text = re.sub(r'\@\w+|\#|<,*?>', '', text)
  text = re.sub(r'[^a-zA-Z\w\s]', '', text)
  text = text.strip()
  return text


In [ ]:
df['Cleaned_text_step1'] = df['Summary'].apply(normalize_text)

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words = set(stopwords.words('english'))
def remove_stopwords(tokens):
  return [word for word in tokens if word not in stop_words]

In [ ]:
df['Filtered_text_step2'] = df['Cleaned_text_step1'].apply(remove_stopwords)

In [ ]:
df


,NEWS (Full Transcript),Summary,Category,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Cleaned_text_step1,Filtered_text_step2
0,what was the principal doing why did he not on...,A group of persons assembles at the hospital t...,crime,NaN,NaN,NaN,NaN,a group of persons assembles at the hospital t...,"[ , g, r, u, p, , f, , p, e, r, n, , e, b, ..."
1,I'm standing right outside the service apartme...,A 19-year-old woman was found brutally murdere...,crime,NaN,NaN,NaN,NaN,a 19yearold woman was found brutally murdered ...,"[ , 1, 9, e, r, l, , w, n, , w, , f, u, n, ..."
2,news that's coming in at the moment the parlia...,The parliament case inquiry and investigation ...,crime,NaN,NaN,NaN,news that's coming in at the moment the parlia...,the parliament case inquiry and investigation ...,"[h, e, , p, r, l, e, n, , c, e, , n, q, u, ..."
3,so let me try and get details of from Amin who...,Amin was a victim of this crime or or supposed...,crime,NaN,NaN,NaN,NaN,amin was a victim of this crime or or supposed...,"[n, , w, , , v, c, , f, , h, , c, r, e, ..."
4,family of Hashi brella who was found dead in t...,Family of Hashi brella who was found dead in t...,crime,NaN,NaN,NaN,NaN,family of hashi brella who was found dead in t...,"[f, l, , f, , h, h, , b, r, e, l, l, , w, ..."
...,...,...,...,...,...,...,...,...,...
195,Indian Prime Minister Narendra Modi launched t...,Prime Minister Narendra Modi launched three su...,tecnology,NaN,NaN,NaN,NaN,prime minister narendra modi launched three su...,"[p, r, e, , n, e, r, , n, r, e, n, r, , , ..."
196,the Pentagon has said that China has expanded ...,There is a need for at least three additional ...,cultural and social affairs,NaN,NaN,NaN,NaN,there is a need for at least three additional ...,"[h, e, r, e, , , , n, e, e, , f, r, , , ..."
197,this is the armir high up in the Himalaya Moun...,The diamir Basha Dam is Pakistan's most expens...,infrastructure and development,NaN,NaN,NaN,NaN,the diamir basha dam is pakistans most expensi...,"[h, e, , r, , b, h, , , , p, k, n, , , ..."
198,China where there are reports that after the C...,There are reports that after the Corona virus ...,health,NaN,NaN,NaN,NaN,there are reports that after the corona virus ...,"[h, e, r, e, , r, e, , r, e, p, r, , h, , ..."


In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize_tokens(tokens):
  return [lemmatizer.lemmatize(word) for word in tokens]

In [ ]:
df['lemmatized_text_step3'] = df['Filtered_text_step2'].apply(lemmatize_tokens)

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L12-v2')
embeddings = model.encode(df['lemmatized_text_step3'].tolist(),show_progress_bar=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
embeddings

array([[-0.01633931,  0.06549253,  0.09462547, ..., -0.00756328,
        -0.09835716,  0.09560324],
       [-0.0184629 , -0.00711819, -0.03135416, ...,  0.07796801,
         0.01386837,  0.01969147],
       [-0.0474512 ,  0.07317099,  0.06171427, ...,  0.02363919,
        -0.05155509,  0.05731367],
       ...,
       [-0.0474512 ,  0.07317099,  0.06171427, ...,  0.02363919,
        -0.05155509,  0.05731367],
       [-0.0474512 ,  0.07317099,  0.06171427, ...,  0.02363919,
        -0.05155509,  0.05731367],
       [-0.00784008, -0.01537755,  0.05500913, ...,  0.03579745,
        -0.04969568,  0.05666394]], dtype=float32)

In [ ]:
from sklearn.cluster import KMeans
num_clusters = 7
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
df['cluster'] = kmeans.fit_predict(embeddings)

In [ ]:
silhouette_avg = silhouette_score(embeddings, df['cluster'])
print(f"Silhouette Score: {silhouette_avg}")

Silhouette Score: 0.4380526840686798


In [ ]:
from sklearn.metrics import davies_bouldin_score

davies_bouldin_index = davies_bouldin_score(embeddings, df['cluster'])

print(f"Davies-Bouldin Score: {davies_bouldin_index}")

Davies-Bouldin Score: 1.1451836977351235


In [ ]:
from sklearn.cluster import AgglomerativeClustering
num_clusters = 7
agglomerative_clustering = AgglomerativeClustering(n_clusters=num_clusters)
df['cluster'] = agglomerative_clustering.fit_predict(embeddings)

In [ ]:
silhouette_avg = silhouette_score(embeddings, df['cluster'])
print(f"Silhouette Score: {silhouette_avg}")

Silhouette Score: 0.45185357332229614


In [ ]:
from sklearn.metrics import davies_bouldin_score

davies_bouldin_index = davies_bouldin_score(embeddings, df['cluster'])

print(f"Davies-Bouldin Score: {davies_bouldin_index}")

Davies-Bouldin Score: 1.393096510249658
